In [1]:
import sys
import torch
import torchvision.transforms as T
from IPython.display import Audio
import librosa
from src.Dataset import AudioMNIST
import custom_transforms as CT
from scipy import signal
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian
from scipy.signal import stft, istft

In [2]:
sample_rate=16000
n_fft=512
root = '../Data'


In [10]:
from custom_transforms import TrimSilence, FixLength
from torch.utils.data import random_split,DataLoader

transforms = [TrimSilence(5), FixLength(16000)]
dataset = AudioMNIST(
    root,
    transform=T.Compose(transforms)
)
batch_size = 1
num_workers = 4
train_size = int(0.8 * len(dataset))
val_size = int(0.10 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True
)


In [11]:
vqvae = torch.load('./saved_models/vqvae_monai.pth').to(device='cuda:0')

/tmp/ipykernel_1118629/2391003069.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vqvae = torch.load('./saved_models/vqvae_monai.pth').to(device='cuda:0')


In [19]:
next(iter(test_loader))[0].shape

torch.Size([1, 2, 128, 44])

In [33]:
x = vqvae.model.index_quantize(next(iter(test_loader))[0].to(device='cuda')).flatten(1)

In [34]:
x.shape

torch.Size([1, 352])

In [35]:
x = x.reshape(1,32,11)

In [36]:
x_hat = vqvae.model.decode_samples(x)

In [37]:
x_hat.shape

torch.Size([1, 2, 128, 44])